In [18]:
from transformers import pipeline


f = open("Greeks.txt","r") 
full_text = f.read()
print(len(full_text))


classifier = pipeline("summarization")
summary=classifier(full_text)


3827


In [19]:
summary_text=summary[0]['summary_text']

In [20]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
from nltk import pos_tag
from nltk import RegexpParser
from nltk import word_tokenize
tokens=word_tokenize(summary_text)
print("After Split:",tokens)
tokens_tag = pos_tag(tokens)
print("After Token:",tokens_tag)
Nouns=[]
for token,tag in tokens_tag:
  if tag.startswith('N'):
    Nouns.append(token)

Nouns

After Split: ['The', 'Nile', 'River', 'fed', 'Egyptian', 'civilization', 'for', 'hundreds', 'of', 'years', '.', 'The', 'ancient', 'Egyptians', 'lived', 'in', 'narrow', 'bands', 'of', 'land', 'on', 'each', 'side', 'of', 'the', 'Nile', '.', 'They', 'called', 'this', 'region', 'the', 'black', 'land', 'because', 'of', 'the', 'fertile', 'soil', 'that', 'the', 'floods', 'deposited', '.', 'Weather', 'in', 'Egypt', 'was', 'almost', 'always', 'the', 'same', '.', 'Eight', 'months', 'of', 'the', 'year', 'were', 'sunny', 'and', 'hot', '.', 'Four', 'months', 'of', 'winter', 'were', 'sunny', 'but', 'cooler', '.']
After Token: [('The', 'DT'), ('Nile', 'NNP'), ('River', 'NNP'), ('fed', 'VBD'), ('Egyptian', 'JJ'), ('civilization', 'NN'), ('for', 'IN'), ('hundreds', 'NNS'), ('of', 'IN'), ('years', 'NNS'), ('.', '.'), ('The', 'DT'), ('ancient', 'JJ'), ('Egyptians', 'NNPS'), ('lived', 'VBN'), ('in', 'IN'), ('narrow', 'JJ'), ('bands', 'NNS'), ('of', 'IN'), ('land', 'NN'), ('on', 'IN'), ('each', 'DT'), ('si

['Nile',
 'River',
 'civilization',
 'hundreds',
 'years',
 'Egyptians',
 'bands',
 'land',
 'side',
 'Nile',
 'region',
 'land',
 'soil',
 'floods',
 'Egypt',
 'Eight',
 'months',
 'year',
 'months',
 'winter',
 'cooler']

In [22]:
from nltk.corpus import wordnet

def synonym_antonym_extractor(phrase):
     
     synonyms =[]
     

     for syn in wordnet.synsets(phrase):
          for l in syn.lemmas():
               synonyms.append(l.name())
               if l.antonyms():
                    synonyms.append(l.antonyms()[0].name())

     return set(synonyms)


similar_words={}
keys=[]
for i in Nouns :
  distractors=synonym_antonym_extractor(phrase=i)
  if len(distractors)>4:
    similar_words[i]=distractors
    keys.append(i)
print(similar_words) 
keys

{'hundreds': {'C', 'one_C', 'century', 'hundred', '100'}, 'years': {'yr', 'long_time', 'year', 'twelvemonth', 'old_age', 'years', 'geezerhood', 'class', 'eld', 'age', 'days'}, 'bands': {'band', 'isthmus', 'stria', 'circle', 'stripe', 'dance_orchestra', 'ring', 'banding', 'lot', 'striation', 'set', 'dance_band'}, 'land': {'body_politic', 'put_down', 'realm', 'domain', 'landed_estate', 'set_ashore', 'terra_firma', 'Edwin_Herbert_Land', 'Din_Land', 'bring_down', 'res_publica', 'state', 'bring', 'shore', 'ground', 'commonwealth', 'demesne', 'land', 'solid_ground', 'kingdom', 'Land', 'earth', 'estate', 'nation', 'set_down', 'acres', 'country', 'farming', 'shoot_down', 'down', 'dry_land', 'soil'}, 'side': {'face', 'slope', 'incline', 'English', 'side', 'position', 'top', 'side_of_meat'}, 'region': {'region', 'neighborhood', 'realm', 'part', 'area'}, 'soil': {'filth', 'land', 'stain', 'grime', 'grease', 'dirt', 'ground', 'territory', 'clean', 'bemire', 'grunge', 'begrime', 'colly', 'dirty', '

['hundreds',
 'years',
 'bands',
 'land',
 'side',
 'region',
 'land',
 'soil',
 'floods',
 'Eight',
 'cooler']

In [23]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
from flashtext import KeywordProcessor
from nltk.tokenize import sent_tokenize

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {} 
    for word in keywords:
        keyword_sentences[word] = []

        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        if len(values)>=1:
          keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(summary_text)
keyword_sentence_mapping = get_sentences_for_keyword(keys, sentences)
        
keyword_sentence_mapping

{'hundreds': ['The Nile River fed Egyptian civilization for hundreds of years .'],
 'years': ['The Nile River fed Egyptian civilization for hundreds of years .'],
 'bands': ['The ancient Egyptians lived in narrow bands of land on each side of the Nile .'],
 'land': ['They called this region the black land because of the fertile soil that the floods deposited .',
  'The ancient Egyptians lived in narrow bands of land on each side of the Nile .'],
 'side': ['The ancient Egyptians lived in narrow bands of land on each side of the Nile .'],
 'region': ['They called this region the black land because of the fertile soil that the floods deposited .'],
 'soil': ['They called this region the black land because of the fertile soil that the floods deposited .'],
 'floods': ['They called this region the black land because of the fertile soil that the floods deposited .'],
 'Eight': ['Eight months of the year were sunny and hot.'],
 'cooler': ['Four months of winter were sunny but cooler .']}

In [27]:
import re
import random

index = 1
print ("------------------------------------------------------------------------------")
print ("                   MCQ GENERATION FROM TEXT                                   ")
print ("--------------------------------------------------------------------------\n\n")
for each in keys:
    choices=[]

    sentence = keyword_sentence_mapping[each][0]
    # print('sentence is ',sentence)
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    
    dups=list(similar_words[each])
    # print(dups)
    choices=dups[:3]
    choices.append(each)
    # choices = [each.capitalize()] + similar_words[each]
    print(choices)
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
          print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", dups[3:20],"\n")
    index = index + 1
    print('correct answers is :',each,'\n')

    if index>10:
      break

------------------------------------------------------------------------------
                   MCQ GENERATION FROM TEXT                                   
--------------------------------------------------------------------------


1) The Nile River fed Egyptian civilization for  _______  of years .
['C', 'one_C', 'century', 'hundreds']
	 a )   hundreds
	 b )   C
	 c )   one_C
	 d )   century

More options:  ['hundred', '100'] 

correct answers is : hundreds 

2) The Nile River fed Egyptian civilization for hundreds of  _______  .
['yr', 'long_time', 'year', 'years']
	 a )   years
	 b )   yr
	 c )   year
	 d )   long_time

More options:  ['twelvemonth', 'old_age', 'years', 'geezerhood', 'class', 'eld', 'age', 'days'] 

correct answers is : years 

3) The ancient Egyptians lived in narrow  _______  of land on each side of the Nile .
['band', 'isthmus', 'stria', 'bands']
	 a )   bands
	 b )   isthmus
	 c )   stria
	 d )   band

More options:  ['circle', 'stripe', 'dance_orchestra', 'r